In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import pandas as pd
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')
df_submission = pd.read_csv('./sample_submission.csv')

In [3]:
df_train

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
from keras.utils import np_utils
Y = np_utils.to_categorical(df_train['label'])
X = df_train.drop(['label'],axis = 1)

Using TensorFlow backend.


In [5]:
import numpy as np
Y = pd.DataFrame(Y)
Y = np.array(Y)
X = np.array(X)

In [6]:
X = np.reshape(X,(X.shape[0],28,28,1))

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.optimizers import Adadelta
from keras.losses import categorical_crossentropy
model = Sequential()
model.add(Conv2D(16, (3,3), padding='same',activation='relu' ,input_shape = (28,28,1)))
model.add(Conv2D(16, (3, 3),padding = 'same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(8, (3, 3),padding = 'same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics=['accuracy'])
model.summary()

W1201 17:41:09.458688 139629449688896 deprecation_wrapper.py:119] From /home/surya/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1201 17:41:12.721206 139629449688896 deprecation_wrapper.py:119] From /home/surya/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1201 17:41:13.988162 139629449688896 deprecation_wrapper.py:119] From /home/surya/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1201 17:41:14.723620 139629449688896 deprecation_wrapper.py:119] From /home/surya/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1201 1

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 16)        2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 8)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 392)               0         
__________

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train,y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42, stratify = Y)

In [9]:
model.save_weights("model.h5")

In [10]:
model.fit(X, Y,
          batch_size=128,
          epochs=12,
          verbose=2,
          validation_data=(x_test, y_test))

W1201 17:41:55.903974 139629449688896 deprecation.py:323] From /home/surya/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 42000 samples, validate on 8400 samples
Epoch 1/12
 - 30s - loss: 0.7328 - acc: 0.8415 - val_loss: 0.1640 - val_acc: 0.9605
Epoch 2/12
 - 25s - loss: 0.1791 - acc: 0.9525 - val_loss: 0.0787 - val_acc: 0.9773
Epoch 3/12
 - 23s - loss: 4.1669 - acc: 0.7253 - val_loss: 8.2261 - val_acc: 0.4893
Epoch 4/12
 - 24s - loss: 8.7404 - acc: 0.4574 - val_loss: 8.3339 - val_acc: 0.4823
Epoch 5/12
 - 24s - loss: 8.1880 - acc: 0.4918 - val_loss: 7.1791 - val_acc: 0.5545
Epoch 6/12
 - 22s - loss: 7.0139 - acc: 0.5646 - val_loss: 5.8848 - val_acc: 0.6346
Epoch 7/12
 - 24s - loss: 6.2093 - acc: 0.6145 - val_loss: 4.6890 - val_acc: 0.7090
Epoch 8/12


KeyboardInterrupt: 

In [ ]:
score = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
predictions = model.predict(np.reshape(np.array(df_test),(df_test.shape[0],28,28,1)))

In [ ]:
result = []
for x in predictions:
    result.append((np.argmax(x)))

In [ ]:
result = np.array(result)

In [ ]:
df_submission['Label'] = result
df_submission

In [ ]:
df_submission.to_csv('./submission_by_surya.csv',index = False)